In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🕶️ 05 - Dose Visualization in VR\n",
    "\n",
    "This notebook demonstrates the rendering of volumetric CT, segmented tumors, and dose distributions in 3D for virtual reality exploration.\n",
    "\n",
    "**Goals**:\n",
    "- Visualize anatomical context + tumor + dose\n",
    "- Use Plotly/VTK for interactive 3D rendering\n",
    "- Optionally export to Unity/WebXR\n",
    "\n",
    "**Input**:\n",
    "- CT volume (DICOM/NIfTI)\n",
    "- Segmented tumor mask\n",
    "- Dose volume (RTDOSE)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Imports\n",
    "import numpy as np\n",
    "import nibabel as nib\n",
    "import plotly.graph_objects as go\n",
    "import matplotlib.pyplot as plt\n",
    "from skimage import measure"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📂 Load Volumes (CT, Tumor, Dose)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load NIfTI volumes\n",
    "ct = nib.load(\"../data/processed/ct_volume.nii.gz\").get_fdata()\n",
    "tumor = nib.load(\"../data/processed/tumor_mask.nii.gz\").get_fdata()\n",
    "dose = nib.load(\"../data/processed/dose_volume.nii.gz\").get_fdata()\n",
    "\n",
    "# Normalize CT for rendering\n",
    "ct = (ct - np.min(ct)) / (np.max(ct) - np.min(ct))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎨 Generate 3D Meshes using Marching Cubes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get iso-surfaces\n",
    "verts_ct, faces_ct, _, _ = measure.marching_cubes(ct, level=0.3)\n",
    "verts_tumor, faces_tumor, _, _ = measure.marching_cubes(tumor, level=0.5)\n",
    "verts_dose, faces_dose, _, _ = measure.marching_cubes(dose, level=np.percentile(dose[dose > 0], 80))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧊 Visualize in Plotly (WebVR-compatible)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def mesh_plot(verts, faces, color, name, opacity=1.0):\n",
    "    x, y, z = verts.T\n",
    "    i, j, k = faces.T\n",
    "    return go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color=color, opacity=opacity, name=name)\n",
    "\n",
    "fig = go.Figure([\n",
    "    mesh_plot(verts_ct, faces_ct, \"lightgray\", \"CT\", 0.1),\n",
    "    mesh_plot(verts_tumor, faces_tumor, \"red\", \"Tumor\", 0.9),\n",
    "    mesh_plot(verts_dose, faces_dose, \"blue\", \"Dose (80%)\", 0.4)\n",
    "])\n",
    "\n",
    "fig.update_layout(\n",
    "    title=\"3D VR Visualization - Tumor and Dose\",\n",
    "    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),\n",
    "    margin=dict(l=0, r=0, b=0, t=30),\n",
    "    showlegend=True\n",
    ")\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ⛓️ Export to OBJ for Unity/VR Platforms (Optional)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from stl import mesh\n",
    "import trimesh\n",
    "\n",
    "tumor_mesh = trimesh.Trimesh(vertices=verts_tumor, faces=faces_tumor)\n",
    "tumor_mesh.export(\"../results/figures/tumor.obj\")\n",
    "\n",
    "# Similarly export CT and Dose meshes if needed"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ Summary\n",
    "- 3D rendering of CT + Tumor + Dose enables immersive analysis\n",
    "- Mesh export can be used in Unity, WebXR, or Three.js for full VR integration\n",
    "- Aligns with radiation dose planning and treatment verification workflows"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": ""
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
